In [1]:
import os
import json
from web3 import Web3, HTTPProvider


import pandas as pd
pd.set_option('display.max_rows', 5000)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("whitegrid")

import datetime

In [2]:
ALCHEMY_API_KEY = os.environ['ALCHEMY_API_KEY']
web3 = Web3(HTTPProvider(f"https://eth-mainnet.alchemyapi.io/v2/{ALCHEMY_API_KEY}"))

In [3]:
pool_addr = "0x3211C6cBeF1429da3D0d58494938299C92Ad5860"
lp_token_addr = "0xdf55670e27bE5cDE7228dD0A6849181891c9ebA1"
tricrypto_addr = "0xD51a44d3FaE010294C616388b506AcdA1bfAAE46"

In [4]:
admin_2 = "0xA464e6DCda8AC41e03616F95f4BC98a13b8922Dc"
admin = "0xeCb456EA5365865EbAb8a2661B0c503410e9B347"

In [5]:
with open("./stgusdc.json", 'r') as f:
    abi = json.load(f)
pool = web3.eth.contract(address=web3.toChecksumAddress(pool_addr), abi=abi)

with open("../tricrypto/tricrypto2.json", 'r') as f:
    abi = json.load(f)
tricrypto2 = web3.eth.contract(address=tricrypto_addr, abi=abi)

In [6]:
start_block = 14405401
current_block = web3.eth.get_block_number()
current_block

14462246

In [7]:
transfer_filter = pool.events.TokenExchange.createFilter(
    fromBlock=start_block,
    toBlock=current_block
)
all_transfers = transfer_filter.get_all_entries()
len(all_transfers)

6061

In [ ]:
swaps = []
for transfer in all_transfers:

    block = transfer['blockNumber']
    tx_hash = transfer['transactionHash'].hex()
    timestamp = datetime.datetime.fromtimestamp(web3.eth.getBlock(block)['timestamp'])

    stg_price = pool.functions.price_oracle().call(block_identifier=block)

    stg_to_usdc_swap = transfer['args']['sold_id'] == 0
    usdc_to_stg_swap = transfer['args']['sold_id'] == 1

    token_sold_decimal = 1e-6
    token_bought_decimal = 1e-18
    if stg_to_usdc_swap:
        token_sold_decimal = 1e-18
        token_bought_decimal = 1e-6

    tokens_sold = transfer['args']['tokens_sold'] * token_sold_decimal
    tokens_bought = transfer['args']['tokens_bought'] * token_bought_decimal

    total_usd_sold = 0
    if usdc_to_stg_swap:
        total_usd_sold = tokens_sold
    total_usd_bought = 0
    if stg_to_usdc_swap:
        total_usd_bought = tokens_bought

    total_stg_sold = 0
    if stg_to_usdc_swap:
        total_stg_sold = tokens_sold
    total_stg_bought = 0
    if usdc_to_stg_swap:
        total_stg_bought = tokens_bought

    total_transaction_usdc = total_usd_bought + total_usd_sold
    total_transaction_usd = total_transaction_usdc * 1

    dict_swaps = {
        'block': block,
        'timestamp': timestamp,
        'stg_price': stg_price,
        'volume_usd': total_transaction_usd,
        'total_usdc_sold': total_usd_sold,
        'total_usdc_bought': total_usd_bought,
        'total_stg_sold': total_stg_sold,
        'total_stg_bought': total_stg_bought,
        'tx_hash': tx_hash
    }
    swaps.append(dict_swaps)

swaps

In [ ]:
df_swaps = pd.DataFrame(swaps)
df_swaps.set_index('timestamp', inplace=True)
df_swaps

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(20, 8))

(df_swaps['volume_usd'] * 1e-6).cumsum().plot(
    ax=ax,
    color='red',
    label='stgusdc',
    lw=4
)

ax.set_ylabel("Swap Volume [million USD]", fontsize=25)
ax.set_xlabel("Block Number", fontsize=25)
ax.tick_params(axis='both', which='major', labelsize=25, colors='black')
ax.yaxis.get_major_formatter().set_scientific(False)
ax.yaxis.get_major_formatter().set_useOffset(False)
ax.legend(fontsize=25);

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(20, 8))

ax.plot(
    df_swaps['timestamp'],
    df_swaps['stg_price'],
    color='black',
    label='Curve.fi STG/USDC',
    lw=4
)

ax.set_ylabel("STG Price [USD]", fontsize=25)
ax.set_xlabel("")
ax.tick_params(axis='both', which='major', labelsize=25, colors='black')
ax.yaxis.get_major_formatter().set_scientific(False)
ax.yaxis.get_major_formatter().set_useOffset(False)
ax.legend(fontsize=25);